# Logistic Regression
## MaSSP 2017, Computer Science

__Chuẩn bị: Xếp Châu__

Hướng dẫn theo tutorial https://www.tensorflow.org/get_started/mnist/pros và một số hình ảnh lấy từ https://www.slideshare.net/ssuser06e0c5/explanation-on-tensorflow-example-deep-mnist-for-expert.

Hãy cùng nhau xây dựng một model phân loại các chữ số viết tay trong MNIST sử dụng mạng lưới CNN như sau:

__Convolutional Layer #1__: Áp dụng 32 filters, kích cỡ 5x5x1, sau đó kích hoạt 32 feature maps thu được với ReLU activation function

__Pooling Layer #1__: Thực hiện max pooling với một filter kích cỡ 2x2 và stride bằng 2 (như vậy các vùng được áp dụng pooling không bị trùng nhau)

__Convolutional Layer #2__: Áp dụng 64 filters, kích cỡ 5x5x32, sau đó là ReLU activation function lên 64 feature maps này

__Pooling Layer #2__: Giống pooling layer #1, Thực hiện max pooling với một filter kích cỡ 2x2 và stride bằng 2

<img src="../../images/TensorFlow_Digit_Classification_with_CNN_Guide/convolution.png" />

__Dense Layer #1__: Với 1024 neurons, sử dụng xác suất dropout 0.4 (xác suất 0.4 cho bất kì neuron nào có thể bị bỏ qua trong một bước training)

__Dense Layer #2 (Logits Layer)__: 10 neurons, mỗi neuron ứng với một nhóm trong 10 nhóm chữ số 0–9.

<img src="../../images/TensorFlow_Digit_Classification_with_CNN_Guide/fc_layer.png" />

## 0. Load data và các hàm trợ giúp
Đầu tiên ta viết các lệnh import quen thuộc và cách load dữ liệu MNIST với $one\_hot=True$.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


Chúng ta sẽ viết 2 hàm $weight\_variable$ và $bias\_variable$ để thu gọn code khi khởi tạo $weight$ và $bias$ cho mỗi lớp trong CNN. Cụ thể, 2 hàm này sẽ nhận vào kích thước và trả về tensor với kích thước tương ứng.

Với $weight$, giá trị của các phần tử thuộc một _distribution_ với _standard deviation_ 0.1. Với $bias$, giá trị ban đầu mỗi phần tử bằng nhau và bằng 0.1.

In [3]:
# Weight Initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

Trong ví dụ này, 2 bước max pooling sử dụng đều filter có kích cỡ 2x2 và giá trị stride là 2, nên ta có thể viết gọn lại thành một hàm $max\_pool\_2x2$ nhận vào input feature maps và trả lại kết quả thu được khi gọi hàm $tf.nn.max\_pool$ với thông số định sẵn.

Tương tự, với 2 bước convolution, ta sử dụng stride bằng 1 và $padding=SAME$ để giữ nguyên chiều dài và chiều rộng của input feature maps. Hàm trợ giúp $conv2d$ nhận input feature maps và filter, trả lại kết quả convolution bằng cách gọi hàm $tf.nn.conv2d$ của TensorFlow.

In [4]:
# Convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# Pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

## 1. Xây dựng model

### 1.1 Inputs
Như các thuật toán khác, ta nhập vào hình ảnh và label tương ứng với các hình ảnh đó.

In [5]:
# Inputs
#x = tf.placeholder(tf.float32, shape=[None, 784])
#y_correct = tf.placeholder(tf.float32, shape=[None, 10])

In [10]:
# Thử với input là 1 ảnh duy nhất từ training set
x = tf.constant(mnist.train.images[0])
y_correct = tf.constant(mnist.train.labels[0])

In [ ]:
sess = tf.Session()

Khác với Deep Neural Network - nhận vào 784 pixel ảnh dưới dạng vector và mất đi tính 2 chiều của ảnh, Convolutional Nueeural Network nhận một hình ảnh 2D vào trong lớp Convolution. Do đó, chúng ta cần "reshape" lại hình ảnh.

_Lưu ý với chiều có giá trị $-1$, điều này nghĩa là TensorFlow sẽ tự tính giá trị phù hợp cho chiều này bằng cách lấy tổng số phần tử chia cho các giá trị đã cung cấp (28, 28, 1)._

In [15]:
x_image = tf.reshape(x, [-1,28,28,1])

In [26]:
sess.run(tf.shape(x_image))

array([ 1, 28, 28,  1])

### 1.2 First Convolutional Layer và Max Pooling

Convolutional Layer #1 Sử dụng 32 filters, mỗi filter có kích cỡ 5x5.

Gọi $W\_conv1$ là ma trận giữ các filter này, ta sẽ dùng hàm $weight\_variable$ ở trên để khởi tạo $W\_conv1$. Như ta đã biết, để khởi tạo $weight$, chúng ta cần cung cấp shape của ma trận này.

Do mỗi filter có kích cỡ 5x5, 2 giá trị đầu tiên của shape của $W\_conv1$ là (5, 5).

Giá trị của chiều thứ 3 trong shape sẽ ứng với số kênh của $x\_image$, trong trường hợp này là 1 (do ảnh là đen trắng).

Giá trị của chiều thứ 4 trong shape tương ứng với số kênh của mỗi ảnh được tạo ra sau bước convolution #1 này. Do chúng ta sử dụng 32 filters, mỗi filter tạo ra một kênh, nên mỗi ảnh ban đầu sẽ tạo 32 kênh.

In [17]:
# First convolutional layer's weight
W_conv1 = weight_variable([5, 5, 1, 32])

Mỗi filter lại có một giá trị bias đi kèm. Do đó 32 filters cần một vector bias có shape [32]

In [18]:
# First convolutional layer's bias
b_conv1 = bias_variable([32])

Sau đó, thực hiện bước convolution:
1. Gọi hàm $conv2d$ định nghĩa ở trên, cho $x\_image$ và $W\_conv1$
2. Cộng bias vào kết quả thu được ở trên
3. Gọi hàm ReLU lên kết quả thu được ở trên, thu được $h\_conv1$ giữ feature maps ở bước Convolutional Layer #1

In [19]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

In [ ]:
sess.run(tf.global_variables_initializer())

In [27]:
sess.run(tf.shape(h_conv1))

array([ 1, 28, 28, 32])

Như đã đề cập ở trên, hàm $conv2d$ chúng ta sử dụng không làm thay đổi kích thước độ dài và độ rộng của mỗi feature map, tuy nhiên số kênh lại tăng lên. Ma trận feature maps $h\_conv1$ có kích thước [số ảnh, 28, 28, 32].

Bước Max Pooling #1 sẽ áp dụng hàm $max\_pool\_2x2$ cho feature maps $h\_conv1$. Kết quả là kích thước của mỗi feature map sẽ bị giảm đi 4 lần (một nửa chiều dài, một nửa chiều rộng). Shape của ma trận feature maps $h\_pool1$ là [số ảnh, 14, 14, 32].

In [28]:
# pooling 
h_pool1 = max_pool_2x2(h_conv1)

In [29]:
sess.run(tf.shape(h_pool1))

array([ 1, 14, 14, 32])

### 1.3 Second Convolutional Layer và Max Pooling

Convolutional Layer #2 sử dụng 64 filter có kích thước dài và rộng là 5x5. Độ sâu của mỗi filter ứng với số kênh của feature maps $h\_pool1$ - 32. Số chiều lớp convolution này tạo ra là 64 - bằng với số filter. Do đó, shape để khởi tạo weight cho bước này $W\_conv2$ là [5, 5, 32, 64].

Tương tự, shape của bias là [64].

Bước Max Pooling #2 giống hệt Max Pooling #1, kích cỡ chiều dài và rộng của mỗi feature map giảm đi một nửa, từ 14x14 còn 7x7. Shape của $h\_pool2$ là [số ảnh, 7, 7, 64].

In [32]:
# Second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [33]:
sess.run(tf.shape(h_pool2))

array([ 1,  7,  7, 64])

Như vậy kết thúc bước Convolution, mỗi hình ảnh lúc đầu có kích thước 28x28 đã trở thành dạng 7x7x64.

### 1.4 Full Connection Layer

$h\_pool2$ chưa phải là input trực tiếp đưa vào FC layer - hay đơn giản là một Neural Network ta đã học. Ta cần "đập dẹt" feature maps này ra để tạo thành một vector cho mỗi ảnh. Hãy nhớ trong DNN, chúng ta nhập vector 784 pixel của mỗi ảnh vào network.

In [34]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

In [35]:
sess.run(tf.shape(h_pool2_flat))

array([   1, 3136])

Sau đó ta xây dựng lớp này tương tự như trong DNN. Trong tutorial này, chúng ta có duy nhất một lớp ẩn với số neuron là 1024. Activation function trong lớp này là ReLU cũng giống lớp Convolution. 

In [36]:
# Fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])    
b_fc1 = bias_variable([1024])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   

In [37]:
sess.run(tf.shape(h_fc1))

array([   1, 1024])

Một điểm khác biệt duy nhất ở đây là ta sử dụng dropout. Với hàm này, ta cung cấp một xác suất giữ lại neuron $keep\_prob$, ví dụ nếu $keep\_prob=0.4$, nghĩa là mỗi neuron sẽ có khả năng 60% bị bỏ qua trong mỗi bước training (kết quả bằng 0).

Hãy tham khảo cheatsheet hoặc documentation của TensorFlow để biết thêm cách sử dụng hàm này.

In [9]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Cuối cùng, ta xây dựng lớp output layer với 10 neuron.

In [10]:
# Output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
 
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

...Đánh giá và bước học trong model.

In [11]:
# Cost function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_correct, logits=y_conv))

# Optimization
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluate model
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_correct,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Chạy model...

In [12]:
# Launch the model
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
file_writer = tf.summary.FileWriter("CNN", sess.graph)
# create a summary for our cost and accuracy
tf.summary.scalar("cost_summary", cross_entropy)
tf.summary.scalar("accuracy", accuracy)
# merge all summaries into a single operation which we can execute in a session 
summary_step = tf.summary.merge_all()
validation_size = 200
for i in range(501):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    if i%50 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch_xs, y_correct: batch_ys, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        valid_xs, valid_ys = mnist.validation.next_batch(validation_size)
        print("Accuracy validation: {}".format(accuracy.eval(
                feed_dict = {x: valid_xs, y_correct: valid_ys,keep_prob: 1.0 })))
    _, summary = sess.run([train_step, summary_step], 
            feed_dict={x: batch_xs, y_correct: batch_ys, keep_prob: 0.6})
    # logging
    file_writer.add_summary(summary, i)
    
print("Accuracy: {}".format(accuracy.eval(feed_dict = {x: mnist.test.images[:validation_size], 
                                               y_correct: mnist.test.labels[:validation_size], 
                                               keep_prob: 1.0})))

step 0, training accuracy 0.1
Accuracy validation: 0.07500000298023224
step 50, training accuracy 0.92
Accuracy validation: 0.8899999856948853
step 100, training accuracy 0.93
Accuracy validation: 0.9100000262260437
step 150, training accuracy 0.91
Accuracy validation: 0.8949999809265137
step 200, training accuracy 0.97
Accuracy validation: 0.9200000166893005
step 250, training accuracy 0.95
Accuracy validation: 0.949999988079071
step 300, training accuracy 0.98
Accuracy validation: 0.925000011920929
step 350, training accuracy 0.98
Accuracy validation: 0.9750000238418579
step 400, training accuracy 0.96
Accuracy validation: 0.9800000190734863
step 450, training accuracy 0.99
Accuracy validation: 0.9700000286102295
step 500, training accuracy 0.98
Accuracy validation: 0.9750000238418579
Accuracy: 0.9900000095367432


## 2. Bài tập
Chạy phần code trên với một nhóm nhỏ các hình ảnh vào theo dõi shape của mỗi bước trung gian.

